In [1]:
import numpy as np
import torch
from longvu.builder import load_pretrained_model
from longvu.constants import (
    DEFAULT_IMAGE_TOKEN,
    IMAGE_TOKEN_INDEX,
)
from longvu.conversation import conv_templates, SeparatorStyle
from longvu.mm_datautils import (
    KeywordsStoppingCriteria,
    process_images,
    tokenizer_image_token,
)
from decord import cpu, VideoReader

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
model_id = "Vision-CAIR/LongVU_Qwen2_7B"

In [3]:
tokenizer, model, image_processor, context_len = load_pretrained_model(
    "Vision-CAIR/LongVU_Qwen2_7B", None, "cambrian_qwen",
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

model.eval()
video_path = "/data/ephemeral/home/MovieclipsAkeelah and the Bee (59) Movie CLIP - Big Words Come From Little Words (2006) HD-_UZxXUwQX84.mp4"
qs = "Describe this video in detail"

vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
fps = float(vr.get_avg_fps())
frame_indices = np.arange(0, len(vr), step=int(fps * 2))
video = []
for frame_index in frame_indices:
    img = vr[frame_index].asnumpy()
    video.append(img)
    
video = np.stack(video)
image_sizes = [video[0].shape[:2]]
video = process_images(video, image_processor, model.config)
video = [item.unsqueeze(0) for item in video]

qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
conv = conv_templates["qwen"].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=video,
        image_sizes=image_sizes,
        do_sample=False,
        temperature=0.2,
        max_new_tokens=1024,
        use_cache=True,
        stopping_criteria=[stopping_criteria],
    )
pred = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [4]:
print(pred)

!The!!!The!!!!
A!!!!
!!!!!
!The!!!!
!!!!!
!!!
!!!
!!!
!!!
!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!!!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!The!!!!
!!
!!!
!!!
!!!
!!!
!!!
!!!!!
!!!
!!!
!The!!!!
!!!!!
!The!!!!
!!!!!
!The!!!
!!!!!
!The!!!
!!!!!
!The!!!
!The!!!
!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!
!!!!!
!The!!!
!!!!!
!The!!!
!The!!
!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!The!!!!
!!!!!
!!!!
!!!!!
!The!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!The!!!
!!!!!
!The!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!
!The!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!The!!!!
!!!!!
!!!!
!!!!!
!The!!!
!The!!!
!The!!!
!!
!!!
!!!
!!!
!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!The!!!!
!The!!!!
!!!!!
!!!!!
!!!!!
!!!!!
!!